# Time Series Analysis ¯\\_(ツ)_/¯
- Python package: `statsmodels`
- References: 
    - https://www.math.u-psud.fr/~goude/Materials/time_series/cours6_ARIMA.pdf
    - http://www.unige.ch/ses/sococ/eda/bernard/box.pdf

## 0. Generalities

#### Stationary Time Series

- Expected value does not depend on time.
- Covariance($X_t$,$X_{t+k}$) depends only on $k$.  

#### Autocovariance Function: 

For a time series $X$ of size $n$, $$ACovF(k)=\frac{1}{n}\sum_{j=1}^{n-k}(X_{j+k}-\mu)(X_j-\mu)$$
$$\mu = \frac{1}{n}\sum_{t=1}^n X_t$$

#### Autocorrelation Function: $$ACorrF(k)=\frac{ACovF(k)}{ACov(0)} = \frac{ACovF(k)}{Cov(X)}$$

#### Partial Autocorrelation Function:  $$PACorrF(k)= Corr\left(X_{t+k}-P_{t,k}(X_{t+k}),X_{t}-P_{t,k}(X_{t})\right) $$  
with $P_{t,k}(X_t)$ the orthogonal projection of $X_t$ onto the linear subspace of Hilbert space spanned by $ x_{t+1},\dots ,x_{t+k}$. 
> PACF not straightforward to compute, there are algorithms for estimating the partial autocorrelation based on the sample autocorrelations (Box, Jenkins, and Reinsel 2008 and Brockwell and Davis, 2009). These algorithms derive from the exact theoretical relation between the partial autocorrelation function and the autocorrelation function. - [source](https://en.wikipedia.org/wiki/Partial_autocorrelation_function)

## 1. ARMA: Autoregressive Moving Average
### About

- Assumes time series is a stationary stochastic process
- ARMA(p,q): $$X_t = \gamma+ \sum_{i=1}^{p}a_iX_{t-i} + \sum_{j=0}^{q} b_i\epsilon_{t-i}$$

With $\epsilon$ being white noise terms, $\delta$ a drift term.

### Fitting ARMA: The **Box-Jenkins** approach:


##### 1. Plot ACF and PACF functions
##### 2. Identify case:
- **Exponential decay of ACF to 0**:
    - Indicates an Autoregressive model (AR(p))
    - Find the order $p$ (value at which PACF is in the **critical region**, a.k.a. statistially 0.  
    
- **One or more spikes, rest  being 0**:
    - Indicates a Moving average model (MA(q))
    - Find the order $q$ (value at which PACF is in the **critical region**, a.k.a. statistially 0.  

- **Exponential decay starting after a few lags**:
    - Indicates a Mixed autoregressive and moving average model.

>**The partial autocorrelation of an AR(p) process is zero at lag $p + 1$ and greater**. If the sample autocorrelation plot indicates that an AR model may be appropriate, then the sample partial autocorrelation plot is examined to help identify the order. **One looks for the point on the PACF plot where the partial autocorrelations for all higher lags are essentially zero**. Placing on the plot an indication of the sampling uncertainty of the sample PACF is helpful for this purpose: this is usually constructed on the basis that the true value of the PACF, at any given positive lag, is zero. This can be formalised as described below.   

>An **approximate test that a given partial correlation is zero (at a 5% significance level)** is given by comparing the sample partial autocorrelations against the **critical region with upper and lower limits given by $\frac{±1.96}{\sqrt(n)}$**, where $n$ is the record length (number of points) of the time-series being analysed. This approximation relies on the assumption that the record length is at least moderately large (say $n$>30) and that the underlying process has finite second moment.  - [source](https://en.wikipedia.org/wiki/Partial_autocorrelation_function)

##### 3. Choose parameters which minify a penalty criterion:
**Akaike's Information Criterion (AIC)**:
$$AIC(p,q)=\frac{1}{T}\sum_{t=1}^{T} $$
 

### Limits of ARMA

- If ACF has no decay to zero or very slow decay, might be because of a trend (i.e. the time series is not stationary)


## 2. ARIMA: Address non-stationary time series with polynomial trend
### About
- Improvement of ARMA when there is a **trend** in the process, making it **non-stationary**:
$$X_t = \sum_{i=1}^{p}a_iX_{t-i} + \sum_{j=0}^{q} b_i\epsilon_{t-i} + \sum_{j=0}^{k} \gamma_t t^k$$ 
- The "I" stands for *integrated*, and this is what's leveraged to manage the trend.
- We build the differentiation operator $$\Delta X_t = X_t - X_{t-1}$$
- We build the k-th order differentiation operator $$\Delta^kX_t = \Delta\left(\Delta^{k-1}X_t\right)$$

- ARIMA(p,k,q): $$\Delta^kX_t = \gamma+   \sum_{i=1}^{p}a_i\Delta^kX_{t-i} + \sum_{j=0}^{q} b_i\epsilon_{t-i}$$

>⚠ By differentiating the stochastic process $X$, we hope that the resulting process will be stationary and thus be approached by ARMA. Indeed, if $X$ presents a polynomial trend of degree $m$, $\Delta^kX$ will present a trend of degree $m-k$


### Theory
We can rewrite:
$$ \left(\Delta^kX_t-\sum_{i=1}^{p}a_i\Delta^kX_{t-i}\right) = \gamma+ \sum_{j=0}^{q} b_i\epsilon_{t-i}$$ 

By introducing the **lag operator $L$**:

$$ \left(1-\sum_{i=1}^{p}a_iL^i\right)\Delta^kX_t = \gamma+ \sum_{j=0}^{q} b_i\epsilon_{t-i}$$ 

Let's assume $\left(1-\sum_{i=1}^{p}a_iL^i\right)$ has a unit root of multiplicity $k$:
$$\Leftrightarrow \Phi(L)\left(1-L\right)^k X_t =    \Theta(L)\epsilon_t$$

It comes:

>$X$ has an ARIMA(p,k,q) representation 
>$$\Leftrightarrow 
\left\{
                \begin{array}{ll}
                  \textrm{There exist } \Phi \textrm{ and } \Theta \textrm{ so that:} \\
                  \Phi(L)\left(1-L\right)^k X_t =    \Theta(L)\epsilon_t\\
                \end{array}
              \right.$$
              
### Fitting ARIMA
&rarr; Again with **Akkaike's Information Criterion (AIC)**, but on the differentied process.
   
### Limits of ARIMA

- If the ACF has **periodical lag spikes**, the process might include seasonality.


## 3. Accounting for polynomial trends & seasonality: SARIMA

### About
- We build the differentiation operator $$\Delta_1 X_t = X_t - X_{t-1}$$
- We build the k-th order differentiation operator $$\Delta_kX_t = X_t - X_{t-k}$$


### Theory
- We follow the same reasoning as in section 2, which yields:

>X has a SARIMA$\left((p,k,q);(P,K,Q)\right)$ representation 
>
>$$\Leftrightarrow 
\left\{
                \begin{array}{ll}
                  \textrm{There exist } \Phi_p, \Phi_P, \Theta_q, \Theta_Q \textrm{ so that:} \\
                  \Phi_p(L)\left(1-L\right)^k\left(1-L^s\right)^K\Phi_P(L^s) X_t =    \Theta_q(L)\Theta_Q(L^s)\epsilon_t\\
                \end{array}
              \right.$$
              
### Fitting SARIMA


## 4. RNNs

Networks with an internal state $h$ (memory), adapted to the study of temporal processes. - [source](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks)

![Alt text](https://stanford.edu/~shervine/teaching/cs-230/illustrations/architecture-rnn.png)


On an NLP task:  

$x^{(t)}$ Batch of characters  
$a^{(t)}$ activation of step t   
$y^{(t)}$ output of layer t (Prediction of upcoming characters)


|Pros | Cons |
|---|--- |
|$W$ are shared across time steps | Aggregating gradient on long sequences is expensive |
|| Vanishing gradients|


**Forward Pass**
$$ y^{(t)} = g_2(W_{ya} a^{(t)})$$
$$ a^{(t+1)} = g_1(W_{aa} a^{(t)} + W_{ax} x^{(t)})$$

**Loss** 


With $y_{k}^{true}(\tau)$ the target value of dimension $d(\tau)$ at step $\tau$, considering a quadratic loss:
    $$ L(\tau) = \frac{1}{2}\sum_{k=1}^{d(\tau)} \left(y_{k}^{true}(\tau) - y_{k}(\tau)\right)^{2} $$


The overall network error over time period $]t', t]$  becomes:
    $$ L^{total}(t',t) = \sum_{\tau=t'+1}^{t} L(\tau) $$

**Gradient**
    $$ \nabla L^{total}(t',t) = \sum_{\tau=t'+1}^{t} \nabla L(\tau) $$

**Parameters Update** - [source](https://stats.stackexchange.com/questions/219914/rnns-when-to-apply-bptt-and-or-update-weights)  

$$ \Delta W = - \eta \nabla_W L^{total}(t',t) $$

> The efficient computation of gradient & parameter update is an active research field and sees various methods:

### Using Real-Time Recurrent Learning
TODO

### Using  Backpropagation Through Time (BTT) on an input sequence defined between $[t_0, t_e]$:



-> To be used if the stream is segmented into sequences independent of each other

**Forward Pass** - Let the network run from step $t_0$ to $t_e$, save entire history of inputs, targets, and network state.

**Single Backward Pass** - Compute $\forall \tau \in [t_0,t_e], \forall k \in [0,d(\tau)] $



With **U the set of indices $l$ such that $w_{lk} \in W_{aa}$**
\begin{equation}
  \delta_k(\tau)=- \frac{\delta L^{total}(t_0,t_e)}{\delta s_k(\tau)} =\left\{
  \begin{array}{@{}ll@{}}
    f_k'(s_k(\tau))e_k(\tau), & \text{if}\ \tau=t_e \\
    f_k'(s_k(\tau))\left( \mathbf{e_k(\tau)} + \sum_{l \in U} w_{lk} \delta_l (\tau+1) \right), & \text{if}\ t_0 \leq \tau \leq t_e
  \end{array}\right.
\end{equation} 
with $s_k(\tau)$ the pre-activation of neuron $k$ at step $\tau$

Then $$\Delta w_{ij} = - \eta \sum_{\tau=t_0+1}^{t_e} \delta_i(\tau) x_j(\tau-1)$$

>This can be viewed as applying canonical backpropagation computation to a feedforward neural network **in which target values are specified in several layers and not just the last one** ! The bolded $e_k(\tau)$  aswell as the sum over the various time steps in the loss are the only RNN-specific components. 

### Using Truncated Backpropagation Through Time (TBTT):

Introduce an update frequency k1, and an aggregation window k2 during gradient computation.

1. **Forward pass** - Step through the next k1 time steps, computing the input, hidden, and output states.
2. **Compute the loss**, i.e. sum only over the k1 time steps.
3. **Backward pass** - Accumulate over the previous k2 time steps (this requires having stored all activations for these time steps).
4. **Update parameters** (this occurs once per chunk of size k1, not incrementally at each time step).
5. If processing multiple chunks of a longer sequence, **store the hidden state at the last time step** (will be used to initialize hidden state for beginning of next chunk). If we've reached the end of the sequence, reset the memory/hidden state and move to the beginning of the next sequence (or beginning of the same sequence, if there's only one).
6. Rinse & **repeat from step 1**.

Gradient computation and updates are performed every k1 time steps because it's computationally cheaper than updating at every time step. Updating multiple times per sequence (i.e. setting k1

less than the sequence length) can accelerate training because weight updates are more frequent.

Backpropagation is performed for only k2
time steps because it's computationally cheaper than propagating back to the beginning of the sequence (which would require storing and repeatedly processing all time steps). Gradients computed in this manner are an approximation to the 'true' gradient computed over all time steps. But, because of the vanishing gradient problem, gradients will tend to approach zero after some number of time steps; propagating beyond this limit wouldn't give any benefit. Setting k2 too short can limit the temporal scale over which the network can learn. However, the network's memory isn't limited to k2 time steps because the hidden units can store information beyond this period (e.g. see Mikolov 2012 and this post).

### Long Short-Term Memory Networks (LSTMs)

## 5.LSTMs

Developed because of the **vanishing/exploding gradient problem** in RNNs:   
Indeed, while backpropagating the errors accross timesteps, we multiply at each timestep our local errors by $w_{lk}$, which makes $\delta_k(\tau) \rightarrow 0/\infty$ for $\tau \rightarrow t_0$

![LSTM principle](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3b/The_LSTM_cell.png/1600px-The_LSTM_cell.png)

C: state  
First: decides what to forget  
Second: decide what to add and in which qty  
Third: decide what to output

## 6. Unobserved Components Models

TODO

## 7. Hierarchical time series:¶